# Quantum Computing Group Project Report
# Time Evolution
### Hugo, Kyran, Lara, Shae, Tamara

In [1]:
import numpy as np

In [2]:
import qiskit 
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp, Statevector

The project is run through qiskit so their packages are required for the code to function.

In [ ]:
def hamiltonian1(L, J):
    """
    Docstring for hamiltonian
    
    :param X: X pauli op
    (X, 0, t)
    :param Y: Y pauli op
    :param Z: Z pauli op
    """
    ham_list = []

    for i in range (int(L)-1):
        ham_list.append(("XX", [i, i+1], -1)) # XX bc Xi and X i+1
        ham_list.append(("YY", [i, i+1], -1))
        ham_list.append(("ZZ", [i, i+1], -Jz))

    H =  SparsePauliOp.from_sparse_list(ham_list, num_qubits=L)

    return H


Jz = 1.1 #can vary

a = 4 #THIS IS L

def initialise(Jz, a):
    """ 
    decides L and the initial state to use depending on the value of Jz
    Returns: L, initial states
    """
    if  Jz > 1:
        L = a
        # all |0> ^tensor L
        psi_0 = Statevector.from_label("0"*L)
        # all |1> ^tensor L
        psi_1 = Statevector.from_label("1"*L)

        return L, [psi_0, psi_1]
    
    elif Jz < -1:
        L = a
        
        alternating = "".join(["10"[(i % 2)] for i in range(L)])
        psi_10 = Statevector.from_label(alternating)
        
        return L, [psi_10]
    
    else:
        raise ValueError("Jz must be >1 or < -1")

L, initial_states = initialise(Jz, a) #L HERE IS WHAT THE TENSOR NUMBER IS

H = hamiltonian1(L, Jz)

print(H)

SparsePauliOp(['IIXX', 'IIYY', 'IIZZ', 'IXXI', 'IYYI', 'IZZI', 'XXII', 'YYII', 'ZZII'],
              coeffs=[-1. +0.j, -1. +0.j, -1.1+0.j, -1. +0.j, -1. +0.j, -1.1+0.j, -1. +0.j,
 -1. +0.j, -1.1+0.j])


In [18]:
#quantum circuit (printed)
qc = QuantumCircuit(int(L)) 

psi = initial_states[0] #chosen an initial state

qc.initialize(psi.data, range(L)) #initialise state

#apply rotation
middle = L //2 #picks L/2 int value
qc.ry(np.pi/2, middle) #rotation here is into x-y plane

evolution_gate = PauliEvolutionGate(H, time=1.0) #apend time evolution
qc.append(evolution_gate, range(int(L)))

print(qc.decompose().draw()) 

          ┌─────────────────────────────────────────────────────┐ ┌──────────┐ »
q_0: ─|0>─┤0                                                    ├─┤0         ├─»
          │                                                     │ │  Rxx(-2) │ »
q_1: ─|0>─┤1                                                    ├─┤1         ├─»
          │  State Preparation(1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) │┌┴──────────┴┐»
q_2: ─|0>─┤2                                                    ├┤ R(π/2,π/2) ├»
          │                                                     │└────────────┘»
q_3: ─|0>─┤3                                                    ├──────────────»
          └─────────────────────────────────────────────────────┘              »
«     ┌──────────┐                                                          »
«q_0: ┤0         ├─■────────────────────────────────────────────────────────»
«     │  Ryy(-2) │ │ZZ(-2.2) ┌──────────┐┌──────────┐                       »
«q_1: ┤1         ├─■─────────┤0      

### Classical case
Runs all options here on jupyter notebook classically

Data is then plotted and analysed

In [ ]:
n = 50 
#will run codes to find optimal solution later

tf = 10 #length of simulation in seconds

t = np.linspace(0, tf, n) #time we run the simulation for

